this notebook builds really pretty transaction graphs.

It uses A LOT of taostats API calls. It was built with a pro accunt in mind. If you are on standard - scale it back, and add times
if you are on standard.. Run it overnight... or upgrade your plan :D

In [ ]:
import requests, json, random
import pandas as pd
import numpy as np
import decimal
import time
from datetime import datetime, timedelta
from rich import print
import os
from dotenv import load_dotenv
#from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
import bittensor as bt

# Load environment variables from .env file
load_dotenv()

api_key= os.getenv('taostats_api')

headers = {
            "accept": "application/json",
            "Authorization": api_key
        }

#colours
ts_teal = '#00DBBC'
ts_red ='#EB5347'
ts_yellow ='#EBC247'
ts_orange ='#FF8B25'
ts_purple ='#D65BB3'
ts_lblue ='#47C6EB'
ts_fusia ='#E82166'
ts_lime ='#BAEB47'
ts_sand ='#EB7347'
ts_dpurple ='#9747FF'
ts_pblue ='#87FFEE'
ts_dgreen ='#007564'
ts_gray ='#909090'
ts_green ='#00B20F'
ts_brown ='#BD7536'
ts_rblue ='#008BDB'


In [ ]:
#get exchanges - we give the exchanges a different colour on the chart
exchange_list = {}
url = "https://api.taostats.io/api/exchange/v1"
response = requests.get(url, headers=headers)
resJson = json.loads(response.text)
exchanges = resJson['data']
for exchange in exchanges:
    ck = exchange['coldkey']['ss58']
    name = exchange['name']
    exchange_list[ck]=name
exchange_list['5FZiuxCBt8p6PFDisJ9ZEbBaKNVKy6TeemVJd1Z6jscsdjib']="kucoin"
print(exchange_list)

In [ ]:
#get current block

url = "https://api.taostats.io/api/block/v1?page=1&limit=1"
response = requests.get(url, headers=headers)
resJson = json.loads(response.text)
current_block = resJson['data'][0]['block_number']
print(current_block)




In [ ]:
#get all transactions from the wallet

def get_transactions(wallet, start, iteration):
    unique_wallets = {}
    page=1
    total_pages =1
    while page <=total_pages:
        url = f"https://api.taostats.io/api/transfer/v1?network=finney&address={wallet}&block_start={start}&block_end={end_block}&limit=200&page={page}"
        response = requests.get(url, headers=headers)
        resJson = json.loads(response.text)
        #if there are results
        if 'pagination' in resJson:
            #set up pagination
            total_pages = resJson['pagination']['total_pages']
            page +=1
            #get the transactions for this wallet
            temp_transactions = resJson['data']
            for temp in temp_transactions:
                buyer = temp['to']['ss58']
                seller = temp['from']['ss58']
                amount = float(temp['amount'])/1e9
                block = temp['block_number']
             
                #is the buyer an exchange?
                if buyer in exchange_list:
                    name = exchange_list[buyer]
                    buyer = name

                #is the seller an exchange?
                if seller in exchange_list:
                    name = exchange_list[seller]
                    seller = name
                
                temp_transaction = {"source":seller,"target":buyer,"amount":amount, "block":block, "iteration":iteration}
                
                # Make a uniqueness key (ignore iteration)
                tx_key = (seller, buyer, amount, block)
                
                if tx_key not in seen_transactions:
                    seen_transactions.add(tx_key)   # mark as seen
                    all_transactions.append(temp_transaction)
                #unique wallet?? 
                for actor in [buyer, seller]:
                    if actor not in all_wallets:
                        all_wallets[actor] = [block]
                    else:
                        all_wallets[actor].append(block)
                    #of the unique buyers, I only need the earliest block of transactions..
                    if actor not in unique_wallets:
                        unique_wallets[actor]=block
                    else:
                        unique_wallets[actor] = min(block, unique_wallets[actor])

                        
    
    return unique_wallets

In [ ]:
#iterate through transactions

def transaction_iteration(wallets, rounds):
    counter = 1
    sources = wallets
    while counter <=rounds:
    
        sources_next = {}
        for wallet, blocks in sources.items():
            block = min(blocks)
  
            if len(wallet) >30:
                #not an exchange name
                #we will use start block on all transactions - i odnt care if they occur before the last one.
                temp_wallets = get_transactions(wallet, start_block, counter)
                
                #add all wallets  to the next seller list
                for temp_wallet, temp_block in temp_wallets.items():
                    if temp_wallet not in sources_next:
                        sources_next[temp_wallet] = [temp_block]
                    else:
                        sources_next[temp_wallet].append(temp_block)

               
        #update the list for the next round and increment the counter
        print(f"iteration {counter}: found {len(all_wallets)} wallets and {len(all_transactions)} transactions.")
        #we've looped through all the sellers in this list
        sources = sources_next
        counter +=1
    #we've looped through each round


In [ ]:
#RPC lets us get all the wallet balances in 1 call - instead of 1 call per walllewt

def get_balances_fast(wallets):
    fast_balances= {}
    taostats_archive_endpoint = f'wss://api.taostats.io/api/v1/rpc/ws/finney_archive?authorization={api_key}'
    taostats_lite_endpoint = f'wss://api.taostats.io/api/v1/rpc/ws/finney_lite?authorization={api_key}'

    not_exchanges = []
    for wallet in wallets:
        if len(wallet)>15:
            not_exchanges.append(wallet)
        else:
            fast_balances[wallet]=0
    
    with bt.Subtensor(network = taostats_lite_endpoint) as subtensor:
        balances = subtensor.get_balances(*(not_exchanges))
        if balances:
            for coldkey, balance in balances.items():
                fast_balances[coldkey] = balance.tao
    return fast_balances


In [ ]:
#one call per wallet - this is slow!

def get_balances(wallets):
    max_retries=3
    backoff_base=1.5
    balances = {}
    session = requests.Session()  # reuse connection

    for wallet in wallets:
        if len(wallet) > 15:
            url = f'https://api.taostats.io/api/account/latest/v1?network=Finney&page=1&limit=1&address={wallet}'
    
            retries = 0
            while retries < max_retries:
                try:
                    resp = session.get(url, headers=headers, timeout=10)
                    resp.raise_for_status()
                    res_json = resp.json()
    
                    if 'data' in res_json and res_json['data']:
                        balance = float(res_json['data'][0]['balance_total']) / 1e9
                    else:
                        balance = 0.0
    
                    balances[wallet] = balance
                    break  # success; exit retry loop
    
                except (requests.exceptions.RequestException, json.JSONDecodeError) as e:
                    retries += 1
                    if retries >= max_retries:
                        print(f"Failed for {wallet} after {max_retries} retries: {e}")
                        balances[wallet] = None  # or 0.0, based on your preference
                    else:
                        delay = backoff_base ** retries + random.uniform(0, 1)
                        time.sleep(delay)
    
            time.sleep(0.1)  # throttle pacing

    return balances





In [ ]:
wallet =main_wallet= "5EA1aXj2sgFEbzCxKJoM9qbiqrmbnLfBqRcpuRHJnsgvdzQ2"
days = 4

end_block = current_block
start_block = current_block-7200*days
start_wallet = {}
start_wallet[wallet] = [start_block]
all_wallets = {}
all_transactions = []
seen_transactions = set()   # holds (seller, buyer, amount, block)


iterations = 7


transaction_iteration(start_wallet, iterations)


print(len(all_transactions),len(all_wallets))

In [ ]:
print(start_block)

In [ ]:
wallet_balances = get_balances_fast(all_wallets)
print(len(wallet_balances))
print(wallet_balances['5DunDrFV6BPbzsgLZUtMQtCq7w9fcUEbM59BUwx5E7N9Bn5E'])

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# Example wallets
wallets = []
for wallet, block in all_wallets.items():
    wallets.append(wallet)

# Example transactions (edges only)
transactions = all_transactions


In [ ]:
from pyvis.network import Network
from ipywidgets import SelectMultiple, Dropdown, Text, IntSlider, interact
from IPython.display import display, IFrame

# --- Define transaction ranges ---
amount_ranges = {
    "1 rao (<1e-8)": (0, 1e-8),
    "0.001 (<0.001)": (1e-8, 0.001),
    "0.1 (<0.1)": (0.001, 0.1),
    "1 (<1)": (0.1, 1),
    "10 (<10)": (1, 10),
    "100 (<100)": (10, 100),
    "1000 (<1000)": (100, 1000),
    ">1000": (1000, float("inf"))
}

wallets_to_skip = [] 
for exchange, name in exchange_list.items():
    wallets_to_skip.append(name)

# ✅ Multi-select checkboxes
amount_selector = SelectMultiple(
    options=list(amount_ranges.keys()), 
    description="Tx Ranges:",
    value=list(amount_ranges.keys())  # start with none selected
)

# --- Extract unique wallets for dropdown ---
wallet_options = ["All"] + list({w for t in transactions for w in [t["source"], t["target"]]})
wallet_selector = Dropdown(options=wallet_options, description="Wallet:")

# --- Slider for iteration ---
max_iter = max(t["iteration"] for t in transactions)
iteration_slider = IntSlider(min=1, max=max_iter, step=1, value=max_iter, description="Max Iteration:")

# --- Text box for highlight wallet ---
highlight_wallet_box = Text(value="", description="Highlight:", placeholder="Enter wallet address")

# --- Main plotting function ---
def plot_network_filtered(amount_range_labels, wallet_filter, highlight_wallet, max_iteration):
    if not amount_range_labels:
        print("⚠️ Please select at least one transaction range")
        return

    # collect all chosen ranges
    selected_ranges = [amount_ranges[label] for label in amount_range_labels]

    net = Network(
        notebook=True, 
        directed=True, 
        cdn_resources="in_line",
        height="1200px", 
        width="1200px"
    )
    net.bgcolor = "#000000"
    # Layout settings
    net.barnes_hut(gravity=-20000, central_gravity=0.3, spring_length=150,
                   spring_strength=0.01, damping=0.09)
    
    # 1️⃣ Filter edges by selected ranges and iteration
    visible_edges = []
    for t in transactions:
        if t["iteration"] <= max_iteration and t['source'] not in wallets_to_skip:
            for (min_amt, max_amt) in selected_ranges:
                if min_amt <= t["amount"] < max_amt:
                    visible_edges.append(t)
                    break  # don’t double-count
    
    # 2️⃣ Filter edges by wallet if selected
    if wallet_filter != "All":
        visible_edges = [
            t for t in visible_edges
            if t["source"] == wallet_filter or t["target"] == wallet_filter
        ]
    
    # 3️⃣ Visible nodes
    visible_nodes = set()
    for t in visible_edges:
        visible_nodes.add(t["source"])
        visible_nodes.add(t["target"])
    
    # 4️⃣ Add nodes
    for w in visible_nodes:
        bal = wallet_balances[w]
        if w == highlight_wallet:
            color, size = ts_teal, 75
        elif w == main_wallet:
            color, size = ts_fusia, 45
        elif w[:1] != "5":  # exchanges
            color, size = ts_lime, 45
        elif bal < 0.01:
            color, size = ts_teal, 15
        elif 0.01 <= bal < 1:
            color, size = ts_orange, 15
        elif 1 <= bal < 10:
            color, size = ts_lblue, 15
        elif 10 <= bal < 100:
            color, size = ts_gray, 15
        else:
            color = ts_dpurple
            if 100 <= bal < 500:
                size = 30
            elif 500 <= bal < 1000:
                size = 45
            else:
                size = 60
    
        net.add_node(w, label=w[:5], title=f"{w}\n{bal}", color=color, size=size)  
    
 
    # 5️⃣ Add edges
    for t in visible_edges:
        edge_kwargs = {
            "value": t["amount"],
            "title": f"Amount: {t['amount']}, Iteration: {t['iteration']}, Block: {t['block']}"
        }
    
        # if dust (1 rao), force edge red
        if t["amount"] == 1e-9:
            edge_kwargs["color"] = "red"
            edge_kwargs["width"] = 4
            edge_kwargs["dashes"] = True 
        net.add_edge(t["source"], t["target"], **edge_kwargs)

    
    # 6️⃣ Display count
    print(f"Showing {len(visible_edges)} txns in {amount_range_labels}, Iteration ≤ {max_iteration}")
    
    # 7️⃣ Show interactive graph
    net.show("wallet_network_filtered.html")
    display(IFrame("wallet_network_filtered.html", width=1200, height=1200))

# --- Combine controls ---
interact(
    plot_network_filtered,
    amount_range_labels=amount_selector,
    wallet_filter=wallet_selector,
    highlight_wallet=highlight_wallet_box,
    max_iteration=iteration_slider
)


In [ ]:
full_wallets = []

for wallet in all_wallets:
    if wallet[:5]  in ["5ESQ2", "5GT6T"]: #"5DunD","5EiXe","5FV99", "5CW8r","5DzsV", "5EEb1", "5HB2Q","5HAkV", "5D1yy", "5ENam", "5GQTY", "5FQEp", "5FnNr", "5HMKQ", "5DHQu"]:
        if wallet not in full_wallets:
            full_wallets.append(wallet)
print(full_wallets)

In [ ]:
from pyvis.network import Network
import networkx as nx
from IPython.display import display, IFrame
from ipywidgets import interact, IntSlider

# --- Configurable wallets ---
wallet_a = "5D1yyu4JrXeCkr4RrFtHbqJNSeKkzNayUFabqMgBu3XFTpcM"
wallet_b = "5DzsVV2L4M9r4uWoyarzPyhfeCv6DDAEs5rM2bpHjmerPcGa"
always_red = wallet_b
always_green = wallet_a

# --- Build full graph ---
G_full = nx.DiGraph()
for t in transactions:
    G_full.add_edge(t["source"], t["target"], amount=float(t["amount"]))

# --- Function to draw limited paths ---
def draw_paths(max_paths=2):
    # Find all simple paths
    all_paths = list(nx.all_simple_paths(G_full, source=wallet_a, target=wallet_b))
    total_paths = len(all_paths)
    
    # Limit paths
    limited_paths = all_paths[:max_paths]
    
    print(f"Total paths found: {total_paths}")
    print(f"Displaying {len(limited_paths)} path(s) (max_paths={max_paths})")
    
    # Create Pyvis network
    net = Network(notebook=True, directed=True, cdn_resources="in_line")
    net.barnes_hut(gravity=-20000, central_gravity=0.3, spring_length=150,
                   spring_strength=0.01, damping=0.09)
    
    added_edges = set()
    
    for path in limited_paths:
        # Add nodes
        for w in path:
            if w == always_red:
                color = "red" 
                size=45
            elif w == always_green:
                color= "green"
                size=15
            else:
                color="blue"
                size=15
            net.add_node(w, label=w[:5], title=w, color=color, size=size)
        # Add edges
        for s, t in zip(path[:-1], path[1:]):
            if (s, t) not in added_edges:
                amount = G_full[s][t]["amount"]
                net.add_edge(s, t, value=amount, title=f"Amount: {amount}",
                             color="orange", width=amount*2)
                added_edges.add((s, t))
    
    # Save and display inline
    net.save_graph("wallet_limited_paths.html")
    display(IFrame("wallet_limited_paths.html", width=700, height=500))

# --- Interactive slider for max_paths ---
max_paths_slider = IntSlider(min=1, max=10, step=1, value=1, description="Max paths:")
interact(draw_paths, max_paths=max_paths_slider)
